# 🔐 VaulteDB Interactive Playground

This notebook demonstrates the core features of VaulteDB — a zero-config, encrypted document database.

You'll see how to create a vault, insert/query documents, inspect raw encrypted content, test vault isolation, and explore secure audit logging and key export.

In [ ]:
# 🧹 Cleanup: Delete old vault files to start fresh for the demo
import os
from pathlib import Path

for base in ["demo_notebook", "vault1", "vault2", "logged_demo"]:
    for ext in [".vault", ".vaultlog"]:
        f = Path(f"{base}{ext}")
        if f.exists():
            f.unlink()

In [ ]:
from vaultedb import vaultedb
from vaultedb.errors import CryptoError
from pprint import pprint
from pathlib import Path

## 🔐 Step 1: Open or Create a Vault

In [ ]:
vault = vaultedb.open("demo_notebook.vault", "vaultedb-notebook-pass")

## ✍️ Step 2: Insert Documents

In [ ]:
vault.insert({"_id": "user-alice", "email": "alice@example.com", "role": "admin"})
vault.insert({"_id": "user-bob", "email": "bob@example.com", "role": "viewer"})

## 🔍 Step 3: Query Encrypted Data

In [ ]:
print("Get by ID:")
pprint(vault.get("user-alice"))

print("\nFind by field (role=admin):")
pprint(vault.find({"role": "admin"}))

## 📦 Step 4: View the Raw Vault File

In [ ]:
print(Path("demo_notebook.vault").read_text()[:500] + "\n...\n(truncated)")

## 🧪 Step 5: Vault Isolation Guarantee

In [ ]:
vault1 = vaultedb.open("vault1.vault", "same-pass")
vault2 = vaultedb.open("vault2.vault", "same-pass")

vault1.insert({"_id": "secret", "msg": "sensitive"})
vault2.store.insert(vault1.store.data["secret"])

try:
    vault2.get("secret")
except CryptoError:
    print("✅ Decryption failed — Vaults are isolated by salt, even with same passphrase.")

## 🪵 Step 6: Enable Encrypted Audit Logging

In [ ]:
vault = vaultedb.open("logged_demo.vault", "hunter2", enable_logging=True)
vault.insert({"_id": "log1", "msg": "audit test"})
vault.get("log1")

log = vault.get_audit_log()
pprint(log.entries())

## 🗝️ Step 7: Export Key + Salt (Secure Backup)

In [ ]:
vault.export_key()

⚠️ **WARNING:** If you lose this key + salt, you will lose access to your data forever. vaultedb does not store your key. Be responsible.

## 🎉 Recap

- Created encrypted vault
- Inserted/queryed secure documents
- Confirmed data is encrypted on disk
- Proved vault isolation
- Logged operations securely
- Exported key for safe backup

**vaultedb: built for privacy-first applications.**